In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('archive')

label_types = os.listdir('archive')
print (label_types) 

['Correct sequence', 'Wrong sequence']


In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('archive' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('archive' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

                tag                                        video_name
0  Correct sequence    archive/Correct sequence/Copy of push up 1.mp4
1  Correct sequence  archive/Correct sequence/Copy of push up 100.mp4
2  Correct sequence  archive/Correct sequence/Copy of push up 101.mp4
3  Correct sequence  archive/Correct sequence/Copy of push up 102.mp4
4  Correct sequence  archive/Correct sequence/Copy of push up 113.mp4
               tag                                     video_name
95  Wrong sequence  archive/Wrong sequence/Copy of push up 55.mp4
96  Wrong sequence  archive/Wrong sequence/Copy of push up 56.mp4
97  Wrong sequence  archive/Wrong sequence/Copy of push up 57.mp4
98  Wrong sequence  archive/Wrong sequence/Copy of push up 58.mp4
99  Wrong sequence  archive/Wrong sequence/Copy of push up 81.mp4


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [13]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [6]:
train_df = pd.read_csv("train.csv")
print(f"Total videos for training: {len(train_df)}")
train_df.sample(10)

Total videos for training: 100


,Unnamed: 0,video_name,tag
4,4,archive/Correct sequence/Copy of push up 113.mp4,Correct sequence
46,46,archive/Correct sequence/Copy of push up 79.mp4,Correct sequence
27,27,archive/Correct sequence/Copy of push up 164.mp4,Correct sequence
21,21,archive/Correct sequence/Copy of push up 144.mp4,Correct sequence
91,91,archive/Wrong sequence/Copy of push up 37.mp4,Wrong sequence
9,9,archive/Correct sequence/Copy of push up 129.mp4,Correct sequence
40,40,archive/Correct sequence/Copy of push up 4.mp4,Correct sequence
39,39,archive/Correct sequence/Copy of push up 3.mp4,Correct sequence
34,34,archive/Correct sequence/Copy of push up 186.mp4,Correct sequence
65,65,archive/Wrong sequence/23.mp4,Wrong sequence


In [7]:
IMG_SIZE=244
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [8]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")
feature_extractor = build_feature_extractor()

In [9]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Correct sequence', 'Wrong sequence']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [10]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [11]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"train_labels in train set: {train_labels.shape}")


Frame features in train set: (100, 20, 2048)
Frame masks in train set: (100, 20)
train_labels in train set: (100, 1)


In [1]:
def get_sequence_model():
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # GRU layers with masking
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    
    # Binary classification output layer
    output = keras.layers.Dense(1, activation="sigmoid")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    # Compile with binary_crossentropy and accuracy as the metric
    rnn_model.compile(
        loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 80
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([train_data[0], train_data[1]], train_labels)
    print(f"Train accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

# Run the experiment with the binary model
_, sequence_model = run_experiment()


NameError: name 'keras' is not defined

In [14]:
model = keras.models.Sequential()
model.add(Conv2D(32, (3,3), activation= 'relu', input_shape = (100,100,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(71, (3, 3), strides=(2,2), padding="same", dilation_rate=(1,1), activation='relu')),

model.add(Conv2D(32, (3,3), activation= 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

C:\Users\Poonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [58]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("train", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

train_video = np.random.choice(train_df["video_name"].values.tolist())
print(f"Train video path: {train_video}")

Train_frames = sequence_prediction(train_video)

Train video path: archive/Correct sequence/Copy of push up 6.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
  Correct sequence: 49.93%
